In [50]:
from pickle import load
import pandas as pd
doc_dp = load(open('/home/jeffmxh/dp_relation/raw_data/output/temp/雅诗兰黛用户微博原创_dpOlay_雅诗兰黛用户微博原创_content_outputltp_loc.pickle','rb'))
#print(doc_dp[doc_dp.content_id==98])
doc_dp = doc_dp.loc[(doc_dp.target_pos!="v") & (doc_dp.source_pos!="v") & (doc_dp.relation!="ADV")]
print(doc_dp)

      target target_pos    source source_pos relation        newtext  \
5         意义          n        非凡          z      ATT           意义非凡   
8         第三          m         次          q      ATT            第三次   
5          这          r        棕瓶          n      ATT            这棕瓶   
6          小          a        棕瓶          n      ATT            小棕瓶   
11        底部         nd       →_→         ws      ATT          底部→_→   
12        那些          r       →_→         ws      ATT          那些→_→   
0         星级          b       咖啡师          n      ATT          星级咖啡师   
1        咖啡师          n        天使          n      ATT          咖啡师天使   
2          小          a        天使          n      ATT            小天使   
9         30          m         份          q      ATT            30份   
1         廣州         ns         在          p      POB            在廣州   
14         我          r        樣子          n      ATT            我樣子   
15         的          u         我          r      RAD           

In [48]:
2>1 and 3>2 and 5>4

True

In [23]:
print(doc_dp.iloc[750, :])

target          两
target_pos      m
source         分钟
source_pos      q
relation      ATT
newtext       两分钟
content_id     99
Name: 7, dtype: object


In [19]:
from pickle import load
import pandas as pd
doc_dp = load(open('/home/jeffmxh/dp_relation/raw_data/output/temp/content_yili_dp美汁源-果粒奶优_伊利优酸乳-天猫旗舰1Tmall评论_content_outputltp_loc.pickle','rb'))
target_list = doc_dp['target']
length_list = list()
for i in range(len(target_list)):
    length = len(target_list.iloc[i])
    length_list.append(length)
    if length>50:
        print(i)
        print(target_list.iloc[i])
print(max(length_list))

26068
8ec9317a4e25546a6374ff1bdd77bf78879bd0a548b8a6c4948adf306795ab7c
64


In [6]:
import os
import re
import pandas as pd
from os import path
import time
import argparse
import pymysql
import logging
import pymysql
import pandas as pd
def get_db_data(query_str):
    conn = pymysql.connect(host='127.0.0.1',port=3306,user='analyzer',password='analyzer@tbs2016',database='dp_relation',charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)
    cur = conn.cursor()
    doc = pd.read_sql_query(query_str, conn)
    doc = pd.DataFrame(doc)
    for column in doc.columns:
        column_null = pd.isnull(doc[column])
        column_null_true = column_null[column_null == True]
        if len(column_null_true) == len(doc):
            del doc[column]
    cur.close()
    conn.close()
    return doc

def get_data_info(project_name,keyword):
    conn = pymysql.connect(host='127.0.0.1',port=3306,user='analyzer',password='analyzer@tbs2016',database='dp_relation',charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)
    cur=conn.cursor()
    sql_query = ("SELECT project_id,keyword_id,info_source_sheet FROM project_key where project_name='%s' and keyword = '%s'"%(project_name,keyword))
    cur.execute(sql_query)
    id_num = cur.fetchall()[0]
    project_id = id_num['project_id']
    keyword_id = id_num['keyword_id']
    table_name = id_num['info_source_sheet']
    return project_id,keyword_id,table_name
    
def readmysql(**kwargs):
    conn = pymysql.connect(host='127.0.0.1',port=3306,user='analyzer',password='analyzer@tbs2016',database='dp_relation',charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)
    cur=conn.cursor()
    doc = pd.read_sql_query("SELECT * FROM %s where project_id=%d and keyword_id = '%s'" %(kwargs['table_name'],kwargs['project_id'],kwargs['keyword_id']), conn)
    doc = pd.DataFrame(doc)
    for column in doc.columns:
        column_null = pd.isnull(doc[column])
        column_null_true = column_null[column_null == True]
        if len(column_null_true) == len(doc):
            del doc[column]
    cur.close()
    conn.close()
    return doc                                                                                                                            

def remove_abnormal_user(project_id, document):
    abnormal_user = get_db_data("SELECT id,origin_id,screen_name FROM weibo_users WHERE project_id=%d AND is_abnormal=1"%project_id)
    if(len(abnormal_user)!=0):
        if(table_name=='weibo_raw_data'):
            document_filter = document.loc[document['user_id'].map(lambda x:x not in list(abnormal_user['id']))]
        elif(table_name=='weibo_messages'):
            document_filter = document.loc[document['weibo_user_origin_id'].map(lambda x:x not in list(abnormal_user['origin_id']))]
        else:
            document_filter = document
        return document_filter

project_id,keyword_id,table_name = get_data_info('华为P10_170511', 'iphone7_京东评论')                                                                                                                                            
document = readmysql(project_id = project_id, table_name = table_name, keyword_id = keyword_id)
document.loc[:,'content_id'] = pd.Series(list(range(len(document))))
# document['content'] = list(map(text_mechanical_removal.main,document['content']))



document.content

0       周日26号就收到手机了，可前天开机激活时，却说无法创建ID，当时怀疑是不是翻新机，由于还没买...
1       已经是第三次在京东买iPhone手机了。前两次手机都很满意，这次也很满意。因为相信京东，所以...
2       又是一个满意的好评。一直对京东很满意。快递小哥也很给力。我下单的精准达。可我把电话卡插错了，...
3       还可以吧，就是包装不怎么好，就是一个盒子，没有什么气垫保护手机，这可是肾换的啊！！开机上官网...
4       不错！第四次在京东买苹果手机了！还是一如既往地给力！赞！！！暖屏也很好我上一个6也是暖屏没毛...
5       手机是不错，但是京东不厚道，退货重新封装当原封机子卖。直接上图了，大家购机留心机身，好多机子...
6       完美完美完美，必须说三遍！外观无瑕疵，通话无电流音，摄像头没有灰尘，全新未激活的完美机器，这...
7       正在试用中，速度很快。未激活。手感不错，一直喜欢苹果。银色的抗造，哈哈哈，希望京东越做越好，...
8       电池经不住用，这个是我的第一个iPhone，感觉也就那么回事，电池还不如我用了快三年的荣耀六...
9       给我妹妹买的，之前用的三星A7，现在手机太卡了，就帮他换了一个！我老妈出了2800，我又出了...
10                      非常喜欢，京东速度非常快。就是希望以后对电子商品的包装打包有些改善
11                        用了一个星期才晒单，全新不是退换机，打官方电话查了才开包装。。
12      来三亚旅游的第二天买的，临时决定的，非常有纪念意义，直接邮寄到了三亚，哈哈。人生中第一部iP...
13      帮朋友代买的，貌似很高大上，京东苹果也开始搞饥饿营销，很好抢预订，发货也快，就是十二期免利息...
14      不错，手机是正品的，*买的，便宜了*块，感谢京东，感谢京东物流，感谢京东快递让我很快拿到手机...
15                4s, 5s都用过，这次的黑7也很不错，秒杀加用卷*买的，不用电信这款很合适！
16      手机差不多用了一个月 是正品 全新无拆封机子 手机也漂亮 使用了一个多月 没有啥毛病 双网通...
17      看着还行。价

In [28]:
import re
ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
illegal_sub = lambda x:ILLEGAL_CHARACTERS_RE.sub('', x)
document['content'].map(illegal_sub)
illegal_match = lambda x:ILLEGAL_CHARACTERS_RE.match(x)
document['content_sub'] = document['content'].map(illegal_sub)
a = document.loc[document['content']!=document['content_sub'],['content','content_sub']]
print(document.content[2358])
print(document.content_sub[2358])
print(a.content_sub)
a
# document['content'][document['content'].apply(illegal_match)]
ILLEGAL_CHARACTERS_RE.findall(str(document.content[2358]))

一只蛐蛐跟猪打赌说：我跳进草里你就看不见我，猪说：我要看得见呢?于是蛐蛐跳进草里。猪在看，猪在看!猪还在看!猪咋还在看呢?  /可爱 提前祝愚人节快乐联想ThinkServer,IBM X86,DELL服务器现货供应电话:15507319197，13973187992 徐君有需要欢迎私聊 诚信第一  期待与您合作 联想服务商支持项目报备授权
一只蛐蛐跟猪打赌说：我跳进草里你就看不见我，猪说：我要看得见呢?于是蛐蛐跳进草里。猪在看，猪在看!猪还在看!猪咋还在看呢?  /可爱 提前祝愚人节快乐联想ThinkServer,IBM X86,DELL服务器现货供应电话:15507319197，13973187992 徐君有需要欢迎私聊 诚信第一  期待与您合作 联想服务商支持项目报备授权
2358    一只蛐蛐跟猪打赌说：我跳进草里你就看不见我，猪说：我要看得见呢?于是蛐蛐跳进草里。猪在看，猪...
Name: content_sub, dtype: object


['\x14']

In [19]:
import gensim
model = gensim.models.Word2Vec.load("/home/jeffmxh/word2vec/weibo.zh.text.model")
model.most_similar(u"足球")

/opt/intel/intelpython35/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


[('乒乓球', 0.7309213280677795),
 ('健儿', 0.7026450634002686),
 ('球员', 0.7012761831283569),
 ('奥运', 0.6894521117210388),
 ('田径', 0.6857425570487976),
 ('男足', 0.6841894388198853),
 ('运动员', 0.6809723973274231),
 ('球队', 0.6800999045372009),
 ('篮球', 0.6778652667999268),
 ('竞技', 0.6769640445709229)]

In [56]:
model.most_similar(u"失意")

[('顺境', 0.8208690881729126),
 ('失之', 0.7815164923667908),
 ('苦痛', 0.7795131802558899),
 ('逆境', 0.7770657539367676),
 ('风平浪静', 0.7738012075424194),
 ('淡然', 0.7691647410392761),
 ('世事', 0.758832573890686),
 ('彷徨', 0.7551109790802002),
 ('所求', 0.7544704675674438),
 ('迷惘', 0.748974084854126)]

In [29]:
model.most_similar(u"失望")

[('失落', 0.5762369632720947),
 ('在意', 0.560211181640625),
 ('难过', 0.5478013753890991),
 ('在乎', 0.5469474196434021),
 ('无能为力', 0.5358082056045532),
 ('患得患失', 0.5323169231414795),
 ('晚一点', 0.5264085531234741),
 ('期望', 0.5228710174560547),
 ('自作多情', 0.517815113067627),
 ('强求', 0.515911877155304)]

In [1]:
import gensim
model = gensim.models.Word2Vec.load("/home/jeffmxh/word2vec/wiki.zh.text.model")
model.most_similar(u"足球")

/opt/intel/intelpython35/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


[('国际足球', 0.5867569446563721),
 ('足球运动', 0.5709316730499268),
 ('篮球', 0.53752201795578),
 ('国家足球队', 0.5170790553092957),
 ('足球比赛', 0.5075417160987854),
 ('足球联赛', 0.5027305483818054),
 ('足球队', 0.5022329688072205),
 ('映美杯', 0.5016276836395264),
 ('板球', 0.49928414821624756),
 ('排球', 0.49457186460494995)]